# DP Vision Experiments Entry Point

Use this notebook to run experiments by selecting the dataset, privacy budgets, and training parameters.
The main entry point is `run_vision_experiment` in `dp_learning.experiment`.

In [ ]:
from dp_learning.experiment import VisionExperimentConfig, run_vision_experiment
from dp_learning.plotting import logs_to_df, plot_acc_vs_epsilon, plot_learning_curve

import pandas as pd


## 1. Configure the experiment grid

Choose a dataset (`emnist`, `mnist`, `cifar10`), methods, privacy budgets, and training parameters.
For DP methods, set `agg_epsilon`/`agg_delta`. For edit-style DP, you can optionally set `bin_epsilon`/`bin_delta`
for random batch sizing.

In [ ]:
dataset_name = "emnist"

methods = [
    "baseline",
    "Hamming-Style DP",
    "Edit-Style DP",
]

agg_epsilons = [0.5, 1.0, 2.0, 4.0]
agg_delta = 1e-5

bin_epsilon = 0.5  # Used when random batching is enabled (Edit-Style DP)
bin_delta = 1e-5

training_params = {
    "batch_size": 256,
    "epochs": 10,
    "lr": 0.1,
    "max_grad_norm": 1.0,
    "dp_mechanism": "Naive",
    "seed": 0,
}


## 2. Run experiments

This section builds configurations for each method and ε value, then runs experiments and stores logs.

In [ ]:
logs = []

for method in methods:
    if method == "baseline":
        config = VisionExperimentConfig(
            dataset_name=dataset_name,
            method=method,
            **training_params,
        )
        _, log = run_vision_experiment(config)
        logs.append(log)
        continue

    for epsilon in agg_epsilons:
        config = VisionExperimentConfig(
            dataset_name=dataset_name,
            method=method,
            agg_epsilon=epsilon,
            agg_delta=agg_delta,
            bin_epsilon=bin_epsilon if method == "Edit-Style DP" else None,
            bin_delta=bin_delta if method == "Edit-Style DP" else None,
            **training_params,
        )
        _, log = run_vision_experiment(config)
        logs.append(log)


## 3. Summarize results

Create a table of final accuracies for easy comparison.

In [ ]:
df = logs_to_df(logs)
df.sort_values(["method", "agg_epsilon"]).reset_index(drop=True)


## 4. Plot accuracy vs privacy budget

Visualize the accuracy trade-off across ε values.

In [ ]:
plot_acc_vs_epsilon(df, dataset=dataset_name)


## 5. Plot a learning curve for a specific setting

Select the method and ε you want to visualize.

In [ ]:
plot_learning_curve(
    logs=logs,
    dataset=dataset_name,
    method="Hamming-Style DP",
    agg_epsilon=1.0,
)
